In [7]:
import pandas as pd
import numpy as np
import sys
sys.path.append('../utilities')
from basic_utilities import *
from model_utilities import *
from pandas.plotting import scatter_matrix
from ml_utilities import *
import scipy.stats as stats
import pdb
import os
import warnings
warnings.filterwarnings("ignore")
import matplotlib.pyplot as plt
import pickle
sys.path.append('../Data_prep')
from data_config import *

In [9]:
!pwd

/Users/stephenzhou/PycharmProjects/fraud_detection/code/fraud_detection/Model


In [11]:
train_test_sample = pd.read_csv('../../../data/processed_data/train_test_sample.csv')

In [ ]:
sk_feature_selection.f_get_interaction_select(raw_data, test_vars, 'fraud_bool', mtype='classification', order=2)